In [7]:
import json, re,  os, time, random 
from bs4 import BeautifulSoup  
import pymongo , bson
import nltk
from nltk.corpus import stopwords
sw=stopwords.words('english')
import pandas as pd 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from datetime import datetime
from config import source

config = {}
with open('../CodeGenerators/config.json', 'r') as f: 
    config=json.loads(f.read())    
url=f'{source}49.page'
url

'https://www.dcurbanmom.com/jforum/forums/show/49.page'

In [8]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn) 
db = client.re
collection = db.posts 

options = Options()
#options.add_argument("--start-minimized")  
driver = webdriver.Chrome(executable_path=config['driver_path'] ,chrome_options=options)  
driver.get(url) 


<ipython-input-8-e38fb2bda14a>:8: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=config['driver_path'] ,chrome_options=options)


In [ ]:
driver.get(url)  

page_topics = driver.find_elements_by_css_selector('.subject-line a') 
hrefs=[]
for page_topic in page_topics[:]:  
    hrefs.append(page_topic.get_attribute('href'))
driver.get(hrefs[3])  


# postrow = driver.find_elements_by_css_selector('.postrow') 




In [ ]:
def get_posts(): 
    postnum = 12 
    soup = BeautifulSoup(driver.page_source, 'html.parser')  
    postinfo = soup.select('div[class*="postinfo"] .genmed a[href*="jforum"]')

    print( len(postinfo) )

    print( re.search('a href="(.*)">', str(postinfo[ postnum ])).groups()[0] )

    dates = soup.select('div[class*="postinfo"] .genmed ')
    print(dates[ postnum ].text)

    postbody = soup.select(' .postbody ')
    print(postbody[ postnum ] ) 




13
/jforum/posts/list/1243387.page#28920623


12/04/2024 13:33

<table class="postbody"><tbody><tr><td>
<input ;"="" class="quote-expand" onclick="if ( this.parentNode.getElementsByTagName('blockquote')[1].style.display != 'block') { var y = this.parentNode.getElementsByTagName('blockquote'); var i; for (i = 0; i &lt; y.length; i++) {y[i].style.display = 'block';}  value = 'Click to hide quotes';} else { var y = this.parentNode.getElementsByTagName('blockquote'); var i; for (i = 1; i &lt; y.length; i++) {y[i].style.display = 'none';}  value = 'Click to show earlier quotes';}" type="button" value="Click to hide quotes"/>
<blockquote style="display: block;">
<div class="posttext">
<cite>Anonymous wrote:</cite><blockquote style="display: block;">
<div class="posttext">
<cite>Anonymous wrote:</cite>This reminds me of the movie "Paris can wait" with Diane Lane. Similar plot. Mrs. Coppola, the producer, based it on a real event on her life that "jolted [her] out of her ennui."
<br/>
<br/> Ba



11/15/2024 00:07


						This board is full of agents and real estate cheerleaders so don’t expect anyone to seriously grapple with the question being posed. 
				


In [ ]:

topics = driver.find_elements(By.CSS_SELECTOR, 'div[class*="postinfo"] .genmed ')

topics[0].text 
topics[0].find_element(By.CSS_SELECTOR, 'a[href*="jforum"] ').get_attribute('href') 

postbody = driver.find_elements(By.CSS_SELECTOR, 'table.postbody ')
postbody[6].get_attribute('innerHTML')

In [ ]:
driver.get(url)     
pages = 0
while True:  
    soup = BeautifulSoup(driver.page_source, 'html.parser')  
    try:
        get_topics( driver.current_url  )
    except e:
        print(e)
            
    nxt = soup.select('link[rel="next"]')  
    if len(nxt) > 0 and pages <= 1: 
        driver.get(f"{base}{nxt[0]['href']}")  
        pages = pages + 1
    else: 
        break

In [ ]:
def get_topics(url):
    driver.get(f"{url}")
    topics = driver.find_elements_by_css_selector('.structItem--thread .structItem-title a') 
    hrefs = []
    for topic in topics[:]:  
        hrefs.append(topic.get_attribute('href'))
    for href in hrefs:  
        driver.get(f"{href}") 
        while True:   
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            print_thread(driver)
            nxt = soup.select('link[rel="next"]')  
            if len(nxt) > 0:   
                driver.get(f"{base}{nxt[0]['href']}") 
                try:
                    print_thread(driver)
                except e:
                    print(e)
            else: 
                break 

In [ ]:
def print_thread(driver): 
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    arts = soup.select('article.message')
    i = 0
    d={}
    for a in arts:  
        d['key']=a['id']
        d['author']=a['data-author']
        d['title']=soup.select('title')[0].get_text() 
        d['url']=base+a.select('header a')[0]['href']
        ele = a.select('time.u-dt') 
        if len(ele) > 0: 
            d['date']=ele[0].get_text() 
        ele = a.select('.message-body .bbWrapper') 
        if len(ele) > 0:  
            d['mess']=ele[0].get_text()
        ele = a.select('a.link--external') 
        if len(ele) > 0:  
            d['mess']=d['mess']+ele[0]['href']
        ele = a.select('.bbImageWrapper') 
        if len(ele) > 0:  
            d['mess']=d['mess']+ele[0]['data-src']
        # print(d)
        # collection.delete_one( d )
        # collection.insert_one( d ) 
        collection.update( { 'key':  d['key']  }, d,  True ) 

In [ ]:
df = pd.DataFrame(list(collection.find()))
df.loc[379].mess

In [ ]:
df.fillna('')
dff = df.loc[df.mess.str.contains('', na=False)] 
for i,r, in dff.iterrows():
    print( r['author'] + ' ' +  r['date']+ ' ' +  r['title'])
    print(r['mess'] )
    print(r['url'] )

In [ ]:
pages = driver.find_elements_by_css_selector('.pageNavWrapper')
for p in pages:
    print(p)

In [ ]:
for e in eles:
    ele=e.find_element_by_css_selector('.structItem-title a')
    print(ele.get_attribute('id'))

In [ ]:
elms = driver.find_elements(By.XPATH, '//ul[contains(@id, "search-results")]//h3/a') 
hrefs = [e.get_attribute('href') for e in elms]  

In [ ]:

for url in hrefs[:12]:
    driver.execute_script( 'window.open();')   
    driver.switch_to_window(driver.window_handles[ len(driver.window_handles)-1])
    driver.get(url)
driver.window_handles[0]